In [3]:
import os
import time
import torch
from PIL import Image
import torch.nn as nn 
import torch.optim as optim 
import torch.nn.functional as F 
import torchvision.transforms as transforms 
from torch.utils.data import DataLoader, Dataset 

In [12]:
def list_to_string_val(flatten_list):
  return " ".join(str(item) for item in flatten_list)

def float32_to_int(num):
    scaled_num = int(round(num * 1000))
    return scaled_num

def extract_feature_map(x):
  feature_map = x.numpy().flatten()
  feature_map_string_values = list_to_string_val(feature_map)

  with open('feature_map_pytorch.txt','wb') as f:
          f.write(str.encode(feature_map_string_values))
          f.write(str.encode("\n"))

In [18]:
class LeNetTinyNet(nn.Module):
    def __init__(self):
        super().__init__()
        
        # onvolutional layers (3,16,32)
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 16, kernel_size=(5, 5), stride=2, padding=1)
        self.conv2 = nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size=(5, 5), stride=2, padding=1)
        self.conv3 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size=(3, 3), padding=1)

        # conected layers
        self.fc1 = nn.Linear(in_features= 64 * 6 * 6, out_features=500)
        self.fc2 = nn.Linear(in_features=500, out_features=50)
        self.fc3 = nn.Linear(in_features=50, out_features=2)


    def forward(self, X):

        X = F.relu(self.conv1(X))
        X = F.max_pool2d(X, 2)
        # extract_feature_map(X)

        X = F.relu(self.conv2(X))
        X = F.max_pool2d(X, 2)

        X = F.relu(self.conv3(X))
        X = F.max_pool2d(X, 2)

        X = X.view(X.shape[0], -1)
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(X))
        X = self.fc3(X)

        return X.log_softmax(dim=-1)


net = LeNetTinyNet()

In [26]:
# Load the trained model
model_path = "model/pytorch_cat_dog_model.pth"
model = LeNetTinyNet()  # Instantiate your CNN model class
model.load_state_dict(torch.load(model_path))
model.eval()

# Preprocess the unseen data
image_path = "Dataset/Dog/4.jpg"
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Apply the necessary transformations
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
image = Image.open(image_path).convert("RGB")
image = transform(image)

image = image.unsqueeze(0)  
print(image)


# Perform inference
with torch.no_grad():
    output = model(image)

# Interpret the model's output
probabilities = torch.softmax(output, dim=1)
predicted_class = torch.argmax(probabilities, dim=1)

# Print the predicted class
print("Predicted Class:", predicted_class.item())


tensor([[[[ 0.2000,  0.2157,  0.2157,  ..., -0.6157, -0.6627, -0.6863],
          [ 0.2471,  0.2235,  0.2314,  ..., -0.5529, -0.5765, -0.6000],
          [ 0.2863,  0.2627,  0.2706,  ..., -0.5294, -0.4980, -0.5373],
          ...,
          [-0.9294, -0.9059, -0.8980,  ..., -0.6392, -0.6784, -0.7176],
          [-0.9137, -0.8902, -0.8980,  ..., -0.6706, -0.7020, -0.7176],
          [-0.9216, -0.8980, -0.8980,  ..., -0.6941, -0.7176, -0.7255]],

         [[ 0.1294,  0.0980,  0.1059,  ..., -0.8039, -0.8118, -0.8275],
          [ 0.1608,  0.1529,  0.1529,  ..., -0.7725, -0.7961, -0.8118],
          [ 0.1922,  0.1922,  0.1843,  ..., -0.7647, -0.7804, -0.7882],
          ...,
          [-0.8431, -0.8196, -0.8353,  ..., -0.6863, -0.7176, -0.7412],
          [-0.8510, -0.8275, -0.8431,  ..., -0.6863, -0.7255, -0.7255],
          [-0.8510, -0.8353, -0.8353,  ..., -0.7020, -0.7255, -0.7255]],

         [[ 0.0667,  0.0745,  0.0431,  ..., -0.8588, -0.8510, -0.8667],
          [ 0.0980,  0.0902,  

OSError: [Errno 28] No space left on device